# Reasearch Track II Assignment 2

---

**Author's Name: Omotoye Shamsudeen Adekoya** <br>
**Student ID: 5066348**

---

## NoteBook Description

This Notebook has two functions; it serves as a user interface for the control of non-holonomic robot in a 3D environment and it also show a graphical feedback of the robot state and behaviour. 

In [5]:
import ipywidgets as widgets
from ipywidgets import Button, Layout, HBox, VBox
from ipywidgets import Button, GridBox, Layout, ButtonStyle
%matplotlib widget

## Robot Control

In [6]:
import matplotlib.pyplot as plt
import rospy
import tf
from nav_msgs.msg import Odometry
import numpy as np
from matplotlib.animation import FuncAnimation
from tf.transformations import euler_from_quaternion
from math import *

class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []

    def plot_init(self):
        self.ax.set_xlim(-5, 5)
        self.ax.set_ylim(-5, 5)
        return self.ln
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    

In [7]:
import rospy
import time
from rt2_assignment2.srv import Command, RandomPosition
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
#from rt2_assignment2.msg import TargetPose 


from all_buttons import *

from ipywidgets import Button, Layout, HBox, VBox

def start_simulation(a):
    ui_client("start")
    start_button.button_style = 'success'
    stop_button.button_style = ''
    
def stop_simulation(b):
    ui_client("stop")
    start_button.button_style = ''
    stop_button.button_style = 'danger'
    
    
def move_forward(a):
    if (rospy.has_param('/linear_velocity')):
        lv_multiplier = rospy.get_param('/linear_velocity')
    else:
        lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*0.5
    pub.publish(vel)
    up_button.button_style = 'info'
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = ''
    
def move_backward(a):
    if (rospy.has_param('/linear_velocity')):
        lv_multiplier = rospy.get_param('/linear_velocity')
    else:
        lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*(-0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = 'info'
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = ''
    
def turn_left(a):
    if (rospy.has_param('/angular_velocity')):
        av_multiplier = rospy.get_param('/angular_velocity')
    else:
        av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(-0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = 'info'
    right_button.button_style = ''
    stop.button_style = ''
    
def turn_right(a):
    if (rospy.has_param('/angular_velocity')):
        av_multiplier = rospy.get_param('/angular_velocity')
    else:
        av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = 'info'
    stop.button_style = ''

def stop_movement(a):
    vel = Twist()
    vel.linear.x = 0.0
    vel.angular.z = 0.0
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = 'danger'
    
def on_value_change(change):
    x = change['new']
    rospy.set_param('/linear_velocity', x)
    
def on_valueang_change(change):
    z = change['new']
    rospy.set_param('/angular_velocity', z)
    

def get_target():
    rospy.wait_for_service('/ui_target')
    try:
        random_target = rospy.ServiceProxy('/ui_target', RandomPosition)
        target = random_target(5.0,-5.0,5.0,-5.0)
    except rospy.ServiceException as e:
        print(f'Service call failed: {e}')
    return target
        
def button_event():
    start_button.on_click(start_simulation)
    stop_button.on_click(stop_simulation)
    up_button.on_click(move_forward)
    down_button.on_click(move_backward)
    left_button.on_click(turn_left)
    right_button.on_click(turn_right)
    stop.on_click(stop_movement)
    a.observe(on_value_change, names='value')
    b.observe(on_valueang_change, names='value')
    
def main():
    rospy.init_node('user_interface')
    button_event()

if __name__ == '__main__':
    rospy.wait_for_service('/user_interface')
    try:
        ui_client = rospy.ServiceProxy('/user_interface', Command)
    except rospy.ServiceException as e:
        print(f'Service call failed: {e}')
    main()
    vis = Visualiser()
    pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
    plt.show(block=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
HBox([start_button, stop_button])

In [9]:
VBox([a, b])

In [10]:
GridBox(children=[up_button,down_button,left_button,right_button, stop],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto auto auto',
            grid_template_columns='16.5% 16.5% 16.5% 16.5% 16.5% 16.5%',
            grid_template_areas='''
            ". up . . . . "
            " left . right . stop ."
            " . down . . . . "
            ''')
       )


GridBox(children=(Button(icon='arrow-up', layout=Layout(grid_area='up', height='50px', width='auto'), style=Bu…

In [47]:
if (rospy.has_param('/reached_target')):
    reached_target = rospy.get_param('/reached_target')
else: 
    reached_target = 0
if (rospy.has_param('/canceled_target')):
    canceled_target = rospy.get_param('/canceled_target')
else:
    canceled_target = 0
if (rospy.has_param('/target_time')):
    time_to_target = rospy.get_param('/target_time')
else:
    time_to_target = []
    
print(f'Reached Target: {reached_target}\nCanceled Target: {canceled_target}')
print(f'The list of time to reach the target: {time_to_target}')
plt.style.use('seaborn-colorblind')
plt.figure(dpi=100)
plt.hist(time_to_target, stacked=True)
plt.figure(dpi=80, figsize=(8,2))
plt.barh(('Reached\nTarget', 'Canceled\nTarget'),(reached_target, canceled_target))
plt.show()

Reached Target: 48
Canceled Target: 5
The list of time to reach the target: [66.26024460792542, 19.526201725006104, 17.14301633834839, 13.924519062042236, 28.046266078948975, 31.119350910186768, 33.02309608459473, 19.409847259521484, 40.691895961761475, 32.800079345703125, 32.28426432609558, 25.534736156463623, 33.18778443336487, 32.54886031150818, 13.3993399143219, 21.43285608291626, 43.10440278053284, 41.720407247543335, 37.876357555389404, 44.81391668319702, 55.40066742897034, 48.007163286209106, 25.006461143493652, 24.57815718650818, 47.78050231933594, 13.108402490615845, 17.889638423919678, 28.579724550247192, 20.28425431251526, 27.080241441726685, 16.053361415863037, 31.54241371154785, 29.6239070892334, 26.318145275115967, 28.739041805267334, 28.562665700912476, 36.87617635726929, 32.661943674087524, 21.7505886554718, 18.868314027786255, 24.69160771369934, 24.154162168502808, 33.24206495285034, 36.6769814491272, 34.149943828582764, 17.129929304122925, 19.87388515472412, 27.179146

/tmp/ipykernel_4998/2889720112.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(dpi=100)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_4998/2889720112.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(dpi=80, figsize=(8,2))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib widget
# x = np.linspace(0.0, 5.0)
# y1 = np.cos(2*np.pi*x)*np.exp(-x)
# y2 = np.cos(2*np.pi*x)
# name = plt.figure(figsize=(8,5), dpi=100)
# name = plt.subplot(211)
# name = plt.plot(x, y1, 'go-')
# name = plt.title('my 2 subplots')
# name = plt.ylabel('Damped')
# name = plt.subplot(212)
# name = plt.plot(x, y2, 'r^-')
# name = plt.xlabel('time (s)')
# name = plt.ylabel('Undamped')
# name = plt.show()


In [43]:
plt.style.available


['seaborn-dark',
 'fast',
 'bmh',
 'fivethirtyeight',
 'seaborn-bright',
 'tableau-colorblind10',
 'dark_background',
 'seaborn-whitegrid',
 'seaborn-notebook',
 'grayscale',
 'seaborn-white',
 'seaborn-colorblind',
 'seaborn-talk',
 'seaborn-deep',
 'classic',
 '_classic_test',
 'seaborn-dark-palette',
 'seaborn-darkgrid',
 'seaborn-pastel',
 'seaborn-paper',
 'seaborn',
 'seaborn-muted',
 'seaborn-ticks',
 'Solarize_Light2',
 'seaborn-poster',
 'ggplot']